In [ ]:
import json

In [ ]:
CONFIG_JSON_PATH = "../config/config.json"

with open(CONFIG_JSON_PATH) as f:
    cfg = json.load(f)

TEST_DATASET_DIR = cfg["dataset"]["test"]
TEST_CROP_DATASET_DIR = cfg["dataset"]["test_crop"]

In [ ]:
import cv2
import os
from collections import Counter
from tqdm import tqdm


def crop_and_resize_fundus_images(image_dir, output_dir, target_size=(320, 320)):
    x_coords = []
    y_coords = []
    widths = []
    heights = []

    for filename in tqdm(os.listdir(image_dir)):
        if filename.endswith((".jpg", ".jpeg", ".png", ".bmp")):
            image_path = os.path.join(image_dir, filename)
            try:
                img = cv2.imread(image_path)
                if img is None:
                    print(f"Error: Could not read image at {image_path}")
                    continue

                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                _, thresh = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY)  # 閾値は適宜調整

                contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                largest_contour = max(contours, key=cv2.contourArea)
                x, y, w, h = cv2.boundingRect(largest_contour)

                x_coords.append(x)
                y_coords.append(y)
                widths.append(w)
                heights.append(h)


            except Exception as e:
                print(f"An error occurred processing {filename}: {e}")

    if not x_coords:
        print("No images were processed.")
        return

    # 最頻値を計算 (複数の最頻値がある場合は、最初の値を使用)
    def most_frequent(List):
        occurence_count = Counter(List)
        return occurence_count.most_common(1)[0][0]

    mode_x = most_frequent(x_coords)
    mode_y = most_frequent(y_coords)
    mode_width = most_frequent(widths)
    mode_height = most_frequent(heights)
    mode_crop_size = min(mode_width, mode_height)
    
    print(f"mode_x: {mode_x}")
    print(f"mode_y: {mode_y}")
    print(f"mode_width: {mode_width}")
    print(f"mode_height: {mode_height}")
    print(f"mode_crop_size: {mode_crop_size}")

    # 各画像を切り抜き、リサイズ、保存
    for filename in tqdm(os.listdir(image_dir)):
        if filename.endswith((".jpg", ".jpeg", ".png", ".bmp")):
            image_path = os.path.join(image_dir, filename)
            try:
                img = cv2.imread(image_path)
                if img is None:
                    continue

                x1 = mode_x
                y1 = mode_y
                x2 = x1 + mode_crop_size
                y2 = y1 + mode_crop_size

                cropped_img = img[y1:y2, x1:x2]

                # リサイズ
                resized_img = cv2.resize(cropped_img, target_size)

                output_path = os.path.join(output_dir, filename)
                cv2.imwrite(output_path, resized_img)


            except Exception as e:
                print(f"An error occurred processing {filename}: {e}")


target_size = (320, 320)  # リサイズ後のサイズ

os.makedirs(TEST_CROP_DATASET_DIR, exist_ok=True)

crop_and_resize_fundus_images(TEST_DATASET_DIR, TEST_CROP_DATASET_DIR, target_size)